In [92]:
class HashTable:

    def __init__(self, max_size):
        '''Create a list with length max_size and all None values'''

        self.data_list = [None for i in range(max_size)]
        self.max_size = max_size


    def get_index(self, name):
        '''Retrive the hash index of a string'''

        result = sum((ord(i)) for i in name)
        return result % len(self.data_list)


    def insert(self, key, new_value, offset=0):
        '''Insert a new key-value pair'''

        idx = self.get_index(key) + offset
        if self.data_list[idx] is not None:
            if self.data_list[idx][0] == key:
                print('There already is such key!')
                return None
            elif idx == self.max_size:
                offset = -idx
                self.insert(key, new_value, offset)
            else:
                offset += 1
                self.insert(key, new_value, offset)
        else:
            self.data_list[idx] = (key, new_value)


    def find(self, key, offset=0):
        '''Given a key, it returns the value associated with it'''

        idx = self.get_index(key) + offset
        if self.data_list[idx] is None:
            print('No such key found! :(')
            return None

        elif self.data_list[idx][0] != key:
            offset += 1
            return self.find(key, offset)
        else:
            return self.data_list[idx][1]


    def update(self, key, new_value):
        '''Uptade the value of a given key'''

        idx = self.get_index(key)
        if self.data_list[idx] is None:
            print('No such key found! :(')
            return None
        self.data_list[idx] = (key, new_value)


    def list_all(self, keys=True, values=True):
        '''List all keys store in the hash table'''
        if keys == True and values == True:
            return [x for x in self.data_list if x is not None]
        
        elif keys == True and values == False:
            return [x[0] for x in self.data_list if x is not None]

        elif keys == False and values == True:
            return [x[1] for x in self.data_list if x is not None]

        else:
            return None
    
    def get_valid_key(self, idx):
        if self.data_list[idx] is not None:
            return self.get_valid_key(idx+1)
        
        else:
            return idx

In [93]:
htable = HashTable(133)

In [94]:
htable.insert('matteo', 83)
htable.insert('oettam', 830)

In [96]:
htable.find('mattoe')

No such key found! :(


In [82]:
htable.get_valid_key(118)

119